<a href="https://colab.research.google.com/github/01Joseph-Hwang10/AI_fundamentals/blob/main/aip2_assign3_20211381.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

housing = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv')

median = housing['total_bedrooms'].median()
housing['total_bedrooms'] = housing['total_bedrooms'].fillna(median)

housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
# del housing['total_rooms']

# housing['rooms_per_household'] = housing['total_rooms'] / housing['households'] # Feature를 이걸 쓰는거랑

housing['bedrooms_per_household'] = housing['total_bedrooms'] / housing['total_rooms'] # Feature를 이걸 쓰는거랑 결과가 조금씩 다릅니다.

del housing['total_rooms'], housing['total_bedrooms']

col_list = list(housing)
col_list.remove("ocean_proximity") # text type
col_list.remove("median_house_value") # target variables needs not to be scaled

# generate a new dataframe that consist of numeric type only
housing_numeric = housing[col_list]
housing_scaled = scaler.fit_transform(housing_numeric)
# Data type conversion from 'Series' to 'DataFrame'
housing_scaled_df = pd.DataFrame(housing_scaled, index=housing_numeric.index, columns=housing_numeric.columns)

# Concatenate
housing = pd.concat([housing_scaled_df, housing["median_house_value"], housing["ocean_proximity"]], axis=1)

housing = pd.get_dummies(housing)

In [18]:
# training - test seperation
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=45)

print('# of train_set : %.0f, # of test_set : %.0f' % (train_set.shape[0], test_set.shape[0]))

# feature and label seperation of training set and test set

train_set_features = train_set.drop('median_house_value', axis=1)
train_set_target = train_set['median_house_value'].copy()

test_set_features = test_set.drop('median_house_value', axis=1)
test_set_target = test_set['median_house_value'].copy()

# of train_set : 16512, # of test_set : 4128


In [20]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Ridge regression
model_ridge = Ridge(alpha = 100)
model_ridge.fit(train_set_features, train_set_target)

print("Train_score : ", model_ridge.score(train_set_features, train_set_target))

# Ridge RMSE
ridge_predicted = model_ridge.predict(test_set_features)
print("Test_score", r2_score(test_set_target, ridge_predicted))

delta = test_set_target - ridge_predicted
print("Errors in housing value prediction : ", np.mean(np.abs(delta) / test_set_target))

final_ridge_mse = mean_squared_error(test_set_target, ridge_predicted)
final_ridge_rmse = np.sqrt(final_ridge_mse)
print('final_ridge_RMSE : ', final_ridge_rmse)

# command to hide the warning box
import warnings
warnings.filterwarnings(action = "ignore")

Train_score :  0.652511581004448
Test_score 0.6333956237676854
Errors in housing value prediction :  0.28415587884460786
final_ridge_RMSE :  69746.54943237505


In [21]:
from sklearn.linear_model import Lasso

# Lasso regression
model_lasso = Lasso(alpha = 1000)
model_lasso.fit(train_set_features, train_set_target)

print("Train_score : ", model_lasso.score(train_set_features, train_set_target))

lasso_predicted = model_lasso.predict(test_set_features)
print("Test_score : ", r2_score(test_set_target, lasso_predicted))

delta = test_set_target - lasso_predicted
print('Errors in housing value prediction : ', np.mean(np.abs(delta) / test_set_target))

final_lasso_mse = mean_squared_error(test_set_target, lasso_predicted)
final_lasso_rmse = np.sqrt(final_lasso_mse)
print("RMSE is : ", final_lasso_rmse)


Train_score :  0.6477400839070926
Test_score :  0.6353348735073737
Errors in housing value prediction :  0.28205827189150406
RMSE is :  69561.83352439076
